In [1]:
import json
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

from sentence_transformers import SentenceTransformer

load_dotenv()

False

In [2]:


with open('../documents.json', 'r') as file:

    docs = json.load(file)
    
    

In [3]:
documents = []
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [4]:
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/python/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '8afcb29781a5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Qt54G1_VQR-xZxT2x_9Q-A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
for doc in documents:
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [8]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
for doc in documents:

    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [11]:
query = "How to run spark in linux"
embedding = model.encode(query)



In [12]:
es_query = {
    "field": "text_vector",
    "query_vector":embedding,
    "k":5,
    "num_candidates":10000

}

In [14]:
response = es_client.search(index = index_name, knn = es_query, source = ["text", "section", "question", "course"])

response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'vDxClJEB9EajuUuhQkXr',
  '_score': 0.8700216,
  '_source': {'question': 'Setting up Java and Spark (with PySpark) on Linux (Alternative option using SDKMAN)',
   'course': 'data-engineering-zoomcamp',
   'section': 'Module 5: pyspark',
   'text': 'Install SDKMAN:\ncurl -s "https://get.sdkman.io" | bash\nsource "$HOME/.sdkman/bin/sdkman-init.sh"\nUsing SDKMAN, install Java 11 and Spark 3.3.2:\nsdk install java 11.0.22-tem\nsdk install spark 3.3.2\nOpen a new terminal or run the following in the same shell:\nsource "$HOME/.sdkman/bin/sdkman-init.sh"\nVerify the locations and versions of Java and Spark that were installed:\necho $JAVA_HOME\njava -version\necho $SPARK_HOME\nspark-submit --version'}},
 {'_index': 'course-questions',
  '_id': '_zxClJEB9EajuUuhSUX3',
  '_score': 0.8319799,
  '_source': {'question': 'Python Kafka: ./build.sh: Permission denied Error',
   'course': 'data-engineering-zoomcamp',
   'section': 'Module 6: streaming with kaf

In [15]:
# Adding a filter

response = es_client.search(index = index_name, knn = es_query, 
                            query = {"match": {"section": "General course-related questions"}},
                            source = ["text", "section", "question", "course"], size = 5)



In [16]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'gTxClJEB9EajuUuhIESF',
  '_score': 10.89992,
  '_source': {'question': 'Course - When will the course start?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."}},
 {'_index': 'course-questions',
  '_id': 'gjxClJEB9EajuUuhIETK',
  '_score': 10.89992,
  '_source': {'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'G